In [1]:
import torch

In [2]:
import pandas as pd

In [3]:
# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [6]:
# Importing the dataset
movies = pd.read_csv('C:\\Nitin\\DeepLearning\\AutoEncoders\\AutoEncoders\\ml-1m\\ml-1m\\movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('C:\\Nitin\\DeepLearning\\AutoEncoders\\AutoEncoders\\ml-1m\\ml-1m\\users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('C:\\Nitin\\DeepLearning\\AutoEncoders\\AutoEncoders\\ml-1m\\ml-1m\\ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [8]:
# Preparing the training set and the test set
training_set = pd.read_csv('C:\\Nitin\\DeepLearning\\AutoEncoders\\AutoEncoders\\ml-100k\\ml-100k\\u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('C:\\Nitin\\DeepLearning\\AutoEncoders\\AutoEncoders\\ml-100k\\ml-100k\\u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [9]:
# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [10]:
def convert(data):
    new_data = []
    for id_user in range(1,nb_users +1):
        id_movies = data[:,1][data[:,0] == id_user]
        id_ratings = data[:,2][data[:,0] == id_user]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return(new_data)    

In [11]:
training_set = convert(training_set)
test_set = convert(test_set)


In [12]:
# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)


In [13]:
#creating the architecture of AE

class SAE(nn.Module):
    def __init__(self,):
        super(SAE,self).__init__()
        self.fc1 = nn.Linear(nb_movies,20)
        self.fc2 = nn.Linear(20,10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self,x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return(x)
    
        

In [14]:
sae = SAE()

In [15]:
criterion = nn.MSELoss()

In [16]:
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [18]:
## training the AE
nb_epoch = 100
for epoch in range(1, nb_epoch+1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        inputVar = Variable(training_set[id_user]).unsqueeze(0)
        target = inputVar.clone()
        if torch.sum(target.data>0)>0:
            output = sae(inputVar)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output,target)
            mean_corrector = nb_movies/float(torch.sum(target.data>0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data[0]*mean_corrector)
            s+=1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: 1.7668000841053235
epoch: 2 loss: 1.0967170089085432
epoch: 3 loss: 1.0536026081360579
epoch: 4 loss: 1.0381494515592569
epoch: 5 loss: 1.0307517400559194
epoch: 6 loss: 1.0264504370898897
epoch: 7 loss: 1.0237513374822484
epoch: 8 loss: 1.0218858920747236
epoch: 9 loss: 1.0207609454727473
epoch: 10 loss: 1.0195077636620062
epoch: 11 loss: 1.018910614654663
epoch: 12 loss: 1.018143282326087
epoch: 13 loss: 1.0177717614686945
epoch: 14 loss: 1.017510501940069
epoch: 15 loss: 1.0170840892605153
epoch: 16 loss: 1.0168604805957877
epoch: 17 loss: 1.0167400625307452
epoch: 18 loss: 1.0163563652393959
epoch: 19 loss: 1.01632153268935
epoch: 20 loss: 1.016132830754786
epoch: 21 loss: 1.0156684228885438
epoch: 22 loss: 1.015779138863319
epoch: 23 loss: 1.0158321576635687
epoch: 24 loss: 1.0155338298447736
epoch: 25 loss: 1.0156586597090769
epoch: 26 loss: 1.0154004257640297
epoch: 27 loss: 1.0150873337510349
epoch: 28 loss: 1.0150018350783054
epoch: 29 loss: 1.012682998317579
ep

In [21]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user])
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data[0]*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: 0.9739286092325107
